PRODUCE MAIN ALBUM TABLE

This code turns out masters database into an SQL-query ready table. This file merges our master database with information gathered from releases and our geocoded studios. Lastly, we merge the coordinates from the geocoder to the urban area boundaries from Kelso et. al

In [ ]:
#%cd path\to\\replication

C:\Users\maxmo\Dropbox\GDS\Dissertation\replication


In [2]:
#SETUP
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point

#INPATH = r"data\masters_with_coords_v1.csv"
INPATH = r"data\masters_with_coords_v3.csv"
OUTPATH = r"data\full_table_v2.csv"
ARTIST_INFO_PATH = r"data\artists_v1.csv"
URBAN_AREAS_PATH = r"data\shapes\stanford-ua-shapefile\yk247bg4748.shp"
COORDS_OUTPATH = r"data\studios_coords_v1.csv"


PROCEEDURE:
1. Keep columns master_id, artist_ids, genres, styles, data_quality, year, title, recorded_id, Name, latitude, longitude
2. drop duplicates
3. drop data_quality < 2 if data_quality ==2 exists for a given master_id
4. spatial merge the lat-lon to an urban area
5. unnest the columns: artist_ids, genres, styles - rename each to the 
6. merge artist level information on artist_id renaming columns 'name' as 'artist_name', 'data_quality' as 'artist_info_data_quality'
7. save to outpath
8. produce a set of unique coordinates, count the number of masters and save the coordinates and the name of the studio to the coordinates path

In [3]:
# Load the data
masters = pd.read_csv(INPATH)
artists = pd.read_csv(ARTIST_INFO_PATH)
urban_areas = gpd.read_file(URBAN_AREAS_PATH)
masters.columns

Index(['master_id', 'artist_ids', 'genres', 'styles', 'data_quality_x', 'year',
       'title', 'release_id', 'recorded_at', 'data_quality_y', 'recorded_id',
       'latitude', 'longitude', 'geocoding_method'],
      dtype='object')

In [4]:
# 1. Keep specified columns
cols_to_keep = ['master_id', 'artist_ids', 'genres', 'styles', 'data_quality_y', 'year', 'title', 'recorded_id', 'latitude', 'longitude', 'geocoding_method']
masters = masters[cols_to_keep]

In [5]:
# 2. Drop duplicates
masters = masters.drop_duplicates()

In [6]:
# 3. Drop data_quality < 2 if data_quality == 2 exists for a given master_id
mask = masters.groupby('master_id')['data_quality_y'].transform('max') == 2
masters = masters[~((masters['data_quality_y'] < 2) & mask)]

In [7]:
# 4. Spatial merge the lat-lon to an urban area
gdf = gpd.GeoDataFrame(
    masters,
    geometry=[Point(xy) for xy in zip(masters.longitude, masters.latitude)],
    crs=urban_areas.crs
)
masters = gpd.sjoin(gdf, urban_areas, how='left', predicate='intersects')

In [8]:
# 5. Unnest the columns: artist_id, genres, styles
import ast
for col in ['artist_ids', 'genres', 'styles']:
    masters[col] = masters[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

masters = masters.explode('artist_ids')
masters = masters.explode('genres')
masters = masters.explode('styles')

# Rename unnested columns to singular
masters = masters.rename(columns={
    'artist_ids': 'artist_id',
    'genres': 'genre',
    'styles': 'style'
})

print(masters['style'].unique())

['Electro' 'Synth-pop' 'Trance' 'Progressive Trance' 'Disco' 'Soft Rock'
 'Pop Rock' 'Grindcore' 'Punk' 'Hardcore' 'Crust' 'Goth Rock' 'Hard Rock'
 'Heavy Metal' 'Blues Rock' 'Garage Rock' 'Indie Rock' 'Jazz-Rock'
 'Prog Rock' 'Reggae' 'Dub' 'Toasting' 'Art Rock' 'Deep House' 'House'
 'Alternative Rock' 'Tribal' 'Folk Rock' 'Acoustic' 'Abstract' 'Ambient'
 'Experimental' 'Minimal' 'Soundtrack' 'Modern Classical' 'Score'
 'New Wave' 'Leftfield' 'Black Metal' 'Field Recording' 'Breaks'
 'Psychedelic Rock' 'Breakbeat' 'Industrial' 'Doom Metal' 'Ska' 'Techno'
 'IDM' 'Avantgarde' 'Ballad' 'J-pop' 'Downtempo' 'Classic Rock' 'Vocal'
 'Glitch' 'EBM' 'Rock & Roll' 'Free Improvisation' 'Spoken Word'
 'Ethereal' 'Post Rock' 'Alternative Metal' 'Contemporary R&B' 'Soul' nan
 'Goregrind' 'Drum n Bass' 'Symphonic Rock' 'Classical'
 'Progressive House' 'Acid Jazz' 'Contemporary' 'Stoner Rock' 'Fusion'
 'Drone' 'Progressive Metal' 'Pop Rap' 'Garage House' 'Glam' 'Europop'
 'Dream Pop' 'Neo-Classical' 

In [9]:
print(masters['genre'].value_counts())

genre
Rock                      274127
Classical                 129473
Pop                       107507
Electronic                 96984
Jazz                       91640
Folk, World, & Country     66654
Funk / Soul                57033
Stage & Screen             26789
Hip Hop                    23606
Blues                      20606
Latin                      19665
Reggae                     17413
Non-Music                   8869
Children's                  3755
Brass & Military            1640
Name: count, dtype: int64


In [10]:
# 6. Merge artist level information on artist_id, renaming columns
artists = artists.rename(columns={'name': 'artist_name', 'data_quality': 'artist_info_data_quality'})
# Ensure both artist_id columns are of the same type (string)
masters['artist_id'] = masters['artist_id'].astype(str)
artists['artist_id'] = artists['artist_id'].astype(str)
masters = masters.merge(artists, left_on='artist_id', right_on='artist_id', how='left')

In [11]:
masters = masters.rename(columns={
    'Name': 'studio_name',
    'name_conve': 'city',
    'ISO_CC': 'ctry_code',
    'max_pop_al': 'est_2010_population'
})

In [12]:
#Find the first year each style appears
first_year_per_style = masters.groupby('style')['year'].min()

# Create new_style_1 column: 1 if row's style's first year and style match, else 0
masters['new_style_1'] = masters.apply(
    lambda row: 1 if pd.notna(row['style']) and row['year'] == first_year_per_style[row['style']] else 0,
    axis=1
)

In [13]:
# Calculate percentage of releases that are a new style per city
city_new_style = masters.groupby('city').agg(
    total_masters=('master_id', 'nunique'),
    new_style_count=('new_style_1', 'sum')
)
city_new_style['pct_new_style'] = city_new_style['new_style_count'] / city_new_style['total_masters']

# Filter cities with more than 500 masters
city_filtered = city_new_style[city_new_style['total_masters'] > 500]

# Top 20 
print("Top 20 cities by % new style:")
print(city_filtered.sort_values('pct_new_style', ascending=False).head(20))

# Bottom 20
print("\nBottom 20 cities by % new style:")
print(city_filtered.sort_values('pct_new_style', ascending=True).head(20))

Top 20 cities by % new style:
           total_masters  new_style_count  pct_new_style
city                                                    
Vienna              1751              748       0.427184
Stuttgart           1019              278       0.272816
Munich              1392              363       0.260776
Kingston1           4817             1189       0.246834
Berlin              4551              961       0.211162
Dresden              551              114       0.206897
Koln                 863              153       0.177289
Sao Paolo            558               98       0.175627
Montreal            1623              277       0.170672
The Hague            668              114       0.170659
Amsterdam           1862              306       0.164339
Antwerpen            526               82       0.155894
Bologna              523               73       0.139579
Budapest            1194              160       0.134003
Dallas               609               79       0.129721
L

In [14]:
from itertools import combinations

# For each master_id, get all style combinations (sorted tuples, size 2)
def get_style_combos(styles):
    styles = list(set(styles))  # remove duplicates
    return list(combinations(sorted(styles), 2))

master_styles = masters.groupby('master_id')['style'].apply(list)

combo_rows = []
for master_id, styles in master_styles.items():
    combos = get_style_combos(styles)
    year = masters.loc[masters['master_id'] == master_id, 'year'].iloc[0]
    for combo in combos:
        combo_rows.append({'master_id': master_id, 'style_combo': combo, 'year': year})

combo_df = pd.DataFrame(combo_rows)

first_year_per_combo = combo_df.groupby('style_combo')['year'].min()

# For each master_id, count how many of its combos are novel (first year for that combo)
def count_novel_combos(row):
    combos = get_style_combos(master_styles[row['master_id']])
    year = row['year']
    return sum(year == first_year_per_combo[combo] for combo in combos)

masters['novel_style_combo_count'] = masters.apply(count_novel_combos, axis=1)

In [15]:
# Calculate average novel_style_combo_count per city
city_novel_combo = masters.groupby('city').agg(
    novel_style_combo_avg=('novel_style_combo_count', 'mean'),
    master_count=('master_id', 'nunique')
).reset_index()

# Filter cities with at least 400 masters
city_novel_combo = city_novel_combo[city_novel_combo['master_count'] >= 400]

# Top 20 cities
top20 = city_novel_combo.sort_values('novel_style_combo_avg', ascending=False).head(20)
print("Top 20 cities by average novel_style_combo_count:")
print(top20)

# Bottom 20 cities
bottom20 = city_novel_combo.sort_values('novel_style_combo_avg', ascending=True).head(20)
print("\nBottom 20 cities by average novel_style_combo_count:")
print(bottom20)

Top 20 cities by average novel_style_combo_count:
             city  novel_style_combo_avg  master_count
526      Montreal               2.938042          1623
835         Tokyo               1.506270          5212
828     The Hague               1.384760           668
133  Buenos Aires               1.203418           523
168       Chicago               0.976638          4248
48         Austin               0.846315          1338
631  Philadelphia               0.821647          2342
78       Berkeley               0.818477          1224
204        Dallas               0.787384           609
884        Vienna               0.782947          1751
328       Houston               0.746082           785
593         Osaka               0.734827           676
398          Koln               0.718800           863
558   New Orleans               0.708333           765
578       Oakland               0.696251          1628
595          Oslo               0.693966          1106
718     San Die

In [16]:
#SAVE CITY COUNTS - USEFUL TO HAVE
city_counts = masters.groupby('city')['master_id'].nunique().reset_index(name='unique_master_count')
city_counts.to_csv(r"data\explorations\city_counts.csv", index=False)

In [17]:
# 7. Save to outpath, dorp bounding box and urban area columns if present
cols_to_drop = [
    'min_bb_xmi', 'max_bb_xmi', 'min_bb_xma', 'max_bb_yma',
    'min_bb_ymi', 'max_bb_ymi', 'min_bb_yma', 'max_bb_yma',
    'mean_bb_xc', 'mean_bb_yc',
    'index_right', 'name_conve', 'max_pop_al', 'max_pop_20', 'max_pop_50',
    'max_pop_30', 'max_pop_31', 'max_natsca', 'min_areakm', 'max_areakm',
    'min_areami', 'max_areami', 'min_perkm', 'max_perkm', 'min_permi', 'max_permi'
]
masters = masters.drop(columns=[col for col in cols_to_drop if col in masters.columns])

masters.to_csv(OUTPATH, index=False)

In [18]:
# 8. Produce a set of unique coordinates, count the number of masters and save
studio_coords = masters.groupby(['Name', 'latitude', 'longitude']).size().reset_index(name='master_count')
studio_coords.to_csv(COORDS_OUTPATH, index=False)

KeyError: 'Name'